In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
from glob import glob
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd 
import seaborn as sns
import matplotlib
from math import pow, sqrt, ceil
from collections import OrderedDict

from sklearn.metrics import accuracy_score, cohen_kappa_score, confusion_matrix

In [3]:
from Analysis.annotations import Annotation
from Analysis.image import Image
from Analysis.expert import Expert, DatasetType, ProjectType

In [4]:
from utils import *

### First Round

In [5]:
path = Path("Results/EIPH/")
files = glob(str(path)+"/*/Participants/*.txt")
gt_path = str(path)+"/FirstRound/GT/ground truth.txt"

users_first_round = {}
users_second_round = {}

ground_truth = Expert("GT", BBType.GroundTruth, DatasetType.EIPH_Exact, ProjectType.Annotation)
ground_truth.add_file(gt_path)

deleted_path = str(path)+"/FirstRound/GT/deleted.txt"
changed_path = str(path)+"/FirstRound/GT/changed.txt"
nms_path = str(path)+"/FirstRound/GT/nms.txt"


deleted_eiph = Expert("deleted", BBType.GroundTruth, DatasetType.EIPH_Exact, ProjectType.Annotation)
deleted_eiph.add_file(deleted_path)

changed_eiph = Expert("changed", BBType.GroundTruth, DatasetType.EIPH_Exact, ProjectType.Annotation)
changed_eiph.add_file(changed_path)

nms_eiph = Expert("changed", BBType.GroundTruth, DatasetType.EIPH_Exact, ProjectType.Annotation)
nms_eiph.add_file(nms_path)

for file_name in files:
    participant = "Expert_" + Path(file_name).stem.split("_")[-1]
    
    if "FirstRound" in file_name:
        expert = Expert(participant, BBType.Detected, DatasetType.EIPH_Exact, ProjectType.Annotation)
        expert.add_file(file_name)
        
        users_first_round[expert.expert] = expert
    else:
        expert = Expert(participant, BBType.Detected, DatasetType.EIPH_Exact, ProjectType.ExpertAlgorithm)
        expert.add_file(file_name)
        
        users_second_round[expert.expert] = expert

In [6]:
ground_truth

GT:  Annos: 237 Seconds: 0.0 Type: EIPH_Exact Project: Annotation

In [7]:
print("First Round")
for user in users_first_round.values():
    print(str(user) + " mIoU: {}".format(user.calc_MIoU(ground_truth)))
    
print("\nSecond Round")    
for user in users_second_round.values():
    print(str(user) + " mIoU: {}".format(user.calc_MIoU(ground_truth)))

First Round
Expert_11:  Annos: 220 Seconds: 57.25 Type: EIPH_Exact Project: Annotation mIoU: 0.4726323367046443
Expert_12:  Annos: 232 Seconds: 68.55 Type: EIPH_Exact Project: Annotation mIoU: 0.4784520856740563
Expert_13:  Annos: 180 Seconds: 55.9 Type: EIPH_Exact Project: Annotation mIoU: 0.458651975765022
Expert_3:  Annos: 213 Seconds: 29.75 Type: EIPH_Exact Project: Annotation mIoU: 0.4010948335671599
Expert_4:  Annos: 236 Seconds: 99.55 Type: EIPH_Exact Project: Annotation mIoU: 0.5981276645470117
Expert_5:  Annos: 190 Seconds: 36.0 Type: EIPH_Exact Project: Annotation mIoU: 0.5147334479279967
Expert_6:  Annos: 199 Seconds: 43.75 Type: EIPH_Exact Project: Annotation mIoU: 0.5335422857216442
Expert_7:  Annos: 212 Seconds: 34.25 Type: EIPH_Exact Project: Annotation mIoU: 0.42720048144293327
Expert_8:  Annos: 197 Seconds: 29.8 Type: EIPH_Exact Project: Annotation mIoU: 0.3879706578366469
Expert_9:  Annos: 182 Seconds: 56.9 Type: EIPH_Exact Project: Annotation mIoU: 0.4483804825988937

In [8]:
expert_names = set([expert for expert in users_first_round] + [expert for expert in users_second_round])
expert_names

{'Expert_11',
 'Expert_12',
 'Expert_13',
 'Expert_3',
 'Expert_4',
 'Expert_5',
 'Expert_6',
 'Expert_7',
 'Expert_8',
 'Expert_9'}

In [9]:
metrics_changed = []
metrics_deleted = []
metrics_nms = []

for expert_name in expert_names:
    
    first_round_expert = users_first_round[expert_name]
    second_round_expert = users_second_round[expert_name] if expert_name in users_second_round else None
    
    total_changed_cells = 0
    total_deleted_cells = 0
    total_nms_cells = 0
    
    first_round_changed_cells = 0
    first_round_deleted_cells = 0
    first_round_nms_cells = 0
    
    second_round_changed_cells = 0
    second_round_deleted_cells = 0
    second_round_nms_cells = 0
    
    if first_round_expert:
        for metric in first_round_expert.calc_metrics(changed_eiph):
            total_changed_cells += int(metric['total positives'])
            first_round_changed_cells += int(metric['total TP'])
            
        for metric in first_round_expert.calc_metrics(deleted_eiph):
            total_deleted_cells += int(metric['total positives'])
            first_round_deleted_cells += int(metric['total TP'])
    
        for metric in user.calc_metrics(nms_eiph):
            total_nms_cells += int(metric['total positives'])
            first_round_nms_cells += int(metric['total TP'])
            
    if second_round_expert:
        for metric in second_round_expert.calc_metrics(changed_eiph):
            second_round_changed_cells += int(metric['total TP'])
            
        for metric in second_round_expert.calc_metrics(deleted_eiph):
            second_round_deleted_cells += int(metric['total TP'])
            
        for metric in second_round_expert.calc_metrics(nms_eiph):
            second_round_nms_cells += int(metric['total TP'])
            
    metrics_changed.append([expert_name, total_changed_cells, first_round_changed_cells, second_round_changed_cells])
    metrics_deleted.append([expert_name, total_deleted_cells, first_round_deleted_cells, second_round_deleted_cells])
    metrics_nms.append([expert_name, total_nms_cells, first_round_nms_cells, second_round_nms_cells])

metrics_changed.append(["Total"] + [sum([row[i] for row in metrics_changed]) for i in range(1, 4)])
metrics_deleted.append(["Total"] + [sum([row[i] for row in metrics_deleted]) for i in range(1, 4)])
metrics_nms.append(["Total"] + [sum([row[i] for row in metrics_nms]) for i in range(1, 4)])

metrics_changed = pd.DataFrame(metrics_changed, columns=['Name','Total Changed', "Annotation", "ExpertAlgorithm"])

metrics_deleted = pd.DataFrame(metrics_deleted, columns=['Name',"Total Deleted", "Annotation", "ExpertAlgorithm"])

metrics_nms = pd.DataFrame(metrics_nms, columns=['Name', "Total NMS", "Annotation", "ExpertAlgorithm"])

D:\ProgProjekte\Python\Object-Detection-Metrics\lib\Evaluator.py:126: RuntimeWarning: invalid value encountered in true_divide
  rec = acc_TP / npos


In [10]:
metrics_changed

,Name,Total Changed,Annotation,ExpertAlgorithm
0,Expert_13,60,37,0
1,Expert_5,60,39,43
2,Expert_3,60,31,0
3,Expert_12,60,34,0
4,Expert_6,60,41,39
5,Expert_9,60,35,0
6,Expert_4,60,44,0
7,Expert_11,60,42,43
8,Expert_7,60,34,0
9,Expert_8,60,22,28


In [11]:
metrics_deleted

,Name,Total Deleted,Annotation,ExpertAlgorithm
0,Expert_13,5,1,0
1,Expert_5,5,0,0
2,Expert_3,5,0,0
3,Expert_12,5,1,0
4,Expert_6,5,2,2
5,Expert_9,5,4,0
6,Expert_4,5,3,0
7,Expert_11,5,2,0
8,Expert_7,5,0,0
9,Expert_8,5,1,0


In [12]:
metrics_nms

,Name,Total NMS,Annotation,ExpertAlgorithm
0,Expert_13,10,6,0
1,Expert_5,10,6,1
2,Expert_3,10,6,0
3,Expert_12,10,6,0
4,Expert_6,10,6,4
5,Expert_9,10,6,0
6,Expert_4,10,6,0
7,Expert_11,10,6,5
8,Expert_7,10,6,0
9,Expert_8,10,6,6
